In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from greenseer.configuration import do_global_configuration
do_global_configuration("config.properties")

- 600519：茅台

In [4]:
from greenseer.dataset.china_dataset import fetch_multi_report
from logging.config import fileConfig
import numpy as np
fileConfig('logging_config.ini')
reports = fetch_multi_report(stock_ids=np.array(["600519","600520"]))
reports.shape

(324, 242)